In [2]:
%load_ext autoreload
%autoreload 2
import torch
from torch.utils import data, model_zoo
import copy
import sys
sys.path.append(sys.path[0] + "/..")
from model.deeplab_multi import DeeplabMulti
import collections
# from torchsummary import summary
import torch.nn as nn
import os
import os.path as osp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Write options
opts = collections.namedtuple("opts", ['use_se', 'train_bn', 'norm_style', 'droprate', \
                           'restore_from'])
opts.use_se = True
opts.train_bn = False
opts.norm_style = 'gn'
opts.droprate = 0.1
opts.restore_from = "./pretrained_weights/SE_GN_batchsize2_1024x512_pp_ms_me0_classbalance7_kl0.1_lr2_drop0.1_seg0.5/GTA5_25000.pth"
opts.num_classes = 6 # TODO change to num_classes after sat and aerial alignment

### Copied from main.py
# Load model
model = DeeplabMulti(num_classes = 19, # pretrained weights original value
                          use_se = opts.use_se, 
                          train_bn = opts.train_bn, 
                          norm_style = opts.norm_style,
                          droprate = opts.droprate, 
                    )
# Restore model
if opts.restore_from[:4] == 'http' :
    saved_state_dict = model_zoo.load_url(opts.restore_from)
else:
    saved_state_dict = torch.load(opts.restore_from, map_location=torch.device('cpu'))

new_params = model.state_dict().copy()
for i in saved_state_dict:
    # Scale.layer5.conv2d_list.3.weight
    i_parts = i.split('.')
    if opts.restore_from[:4] == 'http' :
        if i_parts[1] !='fc' and i_parts[1] !='layer5':
            new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
            print('%s is loaded from pre-trained weight.\n'%i_parts[1:])
    else:
        if i_parts[0] =='module': # when model parallelized
            new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
            print('%s is loaded from pre-trained weight.\n'%i_parts[1:])
        else:
            new_params['.'.join(i_parts[0:])] = saved_state_dict[i]
            print('%s is loaded from pre-trained weight.\n'%i_parts[0:])
model.load_state_dict(new_params)

['conv1', 'weight'] is loaded from pre-trained weight.

['bn1', 'weight'] is loaded from pre-trained weight.

['bn1', 'bias'] is loaded from pre-trained weight.

['bn1', 'running_mean'] is loaded from pre-trained weight.

['bn1', 'running_var'] is loaded from pre-trained weight.

['bn1', 'num_batches_tracked'] is loaded from pre-trained weight.

['layer1', '0', 'conv1', 'weight'] is loaded from pre-trained weight.

['layer1', '0', 'bn1', 'weight'] is loaded from pre-trained weight.

['layer1', '0', 'bn1', 'bias'] is loaded from pre-trained weight.

['layer1', '0', 'bn1', 'running_mean'] is loaded from pre-trained weight.

['layer1', '0', 'bn1', 'running_var'] is loaded from pre-trained weight.

['layer1', '0', 'bn1', 'num_batches_tracked'] is loaded from pre-trained weight.

['layer1', '0', 'conv2', 'weight'] is loaded from pre-trained weight.

['layer1', '0', 'bn2', 'weight'] is loaded from pre-trained weight.

['layer1', '0', 'bn2', 'bias'] is loaded from pre-trained weight.

['layer

<All keys matched successfully>

In [7]:
# Adjust number of class in ASPP classifier heads (layer 5 is aux, layer 6 is final)
for layer in [model.layer5, model.layer6]:
    layer.head[1] = nn.Conv2d(512, opts.num_classes, kernel_size=1, padding=0, dilation=1, bias=False)
    layer.head[1].weight.data.normal_(0, 0.001)
    print("Switched head to %d class" % (opts.num_classes))

torch.save(model.state_dict(), opts.restore_from[:-4] + "_new_head" + ".pth")    

# for name, m in model.named_modules():
#     if name == "layer5.head.1" or name == "layer6.head.1":
#         m = nn.Conv2d(512, opts.num_classes, kernel_size=1, padding=0, dilation=1, bias=False)
#         m.weight.data.normal_(0, 0.001) # Inititalization from deeplab_multi.py
#         print("Switched head of %s to %d" % (name, opts.num_classes))
        
# summary(model, input_size = (3, 1000, 1000))

Switched head to 6
Switched head to 6
